In [1]:
! pip install pymysql
! pip install chromadb
!pip install --upgrade --quiet  langchain langchain-community langchain-experimental

<h1>Initialize LLM </h1>

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key="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",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

<h1>Database Connection</h1>

In [3]:
from langchain.utilities import SQLDatabase

db_host = "localhost"
db_user = "root"
db_password = "keerthi%4098"
db_name = "kap"
db_port=3305

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}",sample_rows_in_table_info=0)

#print(db.table_info)

<h1>Chunck database</h1>

In [4]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

# Initialize ChromaDB client
chroma_client = chromadb.Client(Settings(
    persist_directory="./db_embeddings"  # Path to save embeddings
))

# Create or get the collection
collection_name = "table_embeddings"
collection = chroma_client.get_or_create_collection(name=collection_name)

# Initialize embedding model
MODEL_NAME = "all-mpnet-base-v2"
model = SentenceTransformer(MODEL_NAME)


In [5]:
# Assume db.table_info is a list of table descriptions, one per table
table_metadata = db.table_info.split("\nCREATE TABLE")  # Split based on the "CREATE TABLE" keyword

# Add back the "CREATE TABLE" prefix to each chunk
table_metadata = [f"CREATE TABLE{chunk}" for chunk in table_metadata if chunk.strip()]

# Chunk into groups of 5 tables
table_chunks = [table_metadata[i:i + 5] for i in range(0, len(table_metadata), 5)]

# Process each chunk for embedding and adding to ChromaDB
for idx, chunk in enumerate(table_chunks):
    chunk_description = " ".join(chunk)  # Combine all table schemas in the chunk into a single string
    embedding = model.encode(chunk_description)  # Generate embedding for the chunk
    
    # Add to the ChromaDB collection
    collection.add(
        ids=[f"chunk_{idx}"],  # Unique ID for each chunk
        documents=[chunk_description],  # Document data (merged schema descriptions)
        metadatas=[{"chunk_id": idx}],  # Metadata for identifying the chunk
        embeddings=[embedding]  # Embedding vector
    )


In [6]:
# Function to print entire collection
def print_entire_collection(collection):
    # Retrieve all data from the collection with embeddings included
    data = collection.get(include=["embeddings", "documents", "metadatas"])
    
    # Print the retrieved data
    print("Collection Data:")
    print(data)

# Print the entire collection
print_entire_collection(collection)

Collection Data:
{'ids': ['chunk_0', 'chunk_1', 'chunk_2', 'chunk_3', 'chunk_4', 'chunk_5', 'chunk_6', 'chunk_7', 'chunk_8', 'chunk_9', 'chunk_10', 'chunk_11', 'chunk_12', 'chunk_13', 'chunk_14', 'chunk_15', 'chunk_16', 'chunk_17', 'chunk_18', 'chunk_19', 'chunk_20', 'chunk_21', 'chunk_22', 'chunk_23', 'chunk_24', 'chunk_25', 'chunk_26', 'chunk_27', 'chunk_28', 'chunk_29', 'chunk_30', 'chunk_31', 'chunk_32', 'chunk_33', 'chunk_34', 'chunk_35', 'chunk_36', 'chunk_37', 'chunk_38', 'chunk_39', 'chunk_40', 'chunk_41', 'chunk_42', 'chunk_43', 'chunk_44', 'chunk_45', 'chunk_46', 'chunk_47', 'chunk_48', 'chunk_49', 'chunk_50', 'chunk_51'], 'embeddings': array([[-0.00351168,  0.07843281, -0.04972821, ...,  0.03430319,
         0.00603721, -0.02749244],
       [-0.01279884,  0.04863882, -0.04172581, ...,  0.03995499,
        -0.04600128, -0.02640171],
       [ 0.00881884,  0.05301351, -0.0344341 , ...,  0.0208793 ,
        -0.00644234, -0.00563863],
       ...,
       [-0.03203674,  0.0013323 ,

In [8]:
import re

# Function to extract table names from relevant chunks
def extract_table_names(chunks):
    
    table_pattern = r"CREATE TABLE\s+`?(\w+)`?"  # Regex to match CREATE TABLE statements and capture table name

   
    for line in chunks.split("\n"):  # Split chunk into lines
        match = re.search(table_pattern, line)  # Search for table name in line
        if match:
           
            table_name = match.group(1)  # Extract the table name
            table_names.append(table_name)

    return table_names

In [24]:
# User's query
query_text = "list only the attributes which has maxRange equals to 6 and minrange equals to zero in gradingscalescores with no limiting"

# Generate embedding for the query
query_embedding = model.encode(query_text)

# Retrieve top 3 most relevant chunks
results = collection.query(
    query_embeddings=[query_embedding],  # Embedding of the query
    n_results=3,  # Number of top results to retrieve
    include=["documents", "metadatas"]  # Include relevant metadata and documents
)

# Extract the top chunks
relevant_chunks = results["documents"]  # List of top chunk descriptions
relevant_metadata = results["metadatas"]  # Metadata for the chunks (optional for tracking)

# Print retrieved chunks for debugging
print("Relevant Chunks:")

table_names=[]
for i, chunk in enumerate(relevant_chunks):
    for j,c in enumerate(chunk):
        print(f"Chunk {j + 1}:\n{c}\n")
        table_names.extend(extract_table_names(c))
        


Relevant Chunks:
Chunk 1:
CREATE TABLE questiongradinggroups (
	`questionGradingGroupId` INTEGER UNSIGNED NOT NULL COMMENT 'Auto-increment ID for the question grading category groups' AUTO_INCREMENT, 
	name VARCHAR(255) COMMENT 'name of the grading category group', 
	description VARCHAR(255) COMMENT 'description of the grading category group', 
	`isPublished` TINYINT UNSIGNED NOT NULL COMMENT 'Whether grading category group is published or not' DEFAULT '0', 
	`isArchived` TINYINT UNSIGNED NOT NULL COMMENT 'Whether grading category group is archived or not' DEFAULT '0', 
	`createdAt` TIMESTAMP NOT NULL COMMENT 'time the record was created' DEFAULT CURRENT_TIMESTAMP, 
	`createdBy` INTEGER UNSIGNED NOT NULL COMMENT 'personId of the account that created the record', 
	`updatedAt` TIMESTAMP NOT NULL COMMENT 'time the record was last updated' DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP, 
	`updatedBy` INTEGER UNSIGNED NOT NULL COMMENT 'personId of the account that last updated the r

In [21]:
table_names=set(table_names)
table_names

{'contentmodulelinks',
 'contents',
 'contenttypes',
 'contentucms',
 'dashboardviews',
 'dimensiontypes',
 'discussionmedia',
 'discussionmessages',
 'discussionnodes',
 'discussionofferings',
 'questiontypes',
 'questionucms',
 'refreshtokens',
 'remotepersons',
 'scormstatistics'}

In [10]:
len(table_names)

15

In [23]:
from langchain_experimental.sql import SQLDatabaseChain

# Create a filtered SQLDatabase object
filtered_db = SQLDatabase.from_uri(
    f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}",
    include_tables=table_names,  # Only include relevant tables
    sample_rows_in_table_info=0  # Disable sampling for speed
)

db_chain = SQLDatabaseChain.from_llm(llm, filtered_db, verbose=True)
response = db_chain.run(query_text)
if response.strip().startswith('```'):
        # Extract just the SQL query from between the markdown code blocks
        response = response.split('```')[1]
        if response.startswith('sql'):
            response = response[3:]
        response = response.strip()
print(response)



> Entering new SQLDatabaseChain chain...
what are different contenttypes
SQLQuery:Question: what are different contenttypes
SQLQuery: 
```sql
SELECT DISTINCT `name` 
FROM `contenttypes` 
ORDER BY `name` 
LIMIT 5;
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT DISTINCT `name` \nFROM `contenttypes` \nORDER BY `name` \nLIMIT 5;\n``' at line 1")
[SQL: ```sql
SELECT DISTINCT `name` 
FROM `contenttypes` 
ORDER BY `name` 
LIMIT 5;
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

<h1>Retrived chunks</h1>